In [ ]:
import pandas as pd
from datetime import date
import os

In [ ]:
tDay=date.today()

In [ ]:
quote='quote borrow: 1709-HK / 2598-HK / 1783-HK / 2522-HK / 2473-HK / 2477-HK / 2490-HK / 2495-HK / 2582-HK / 9636-HK / 7226-HK / 7552-HK / 600418-CH / 600104-CH'

In [ ]:
text='''
Huatai HT cover 295.5K shares 601988-C1 limit 5.35, DMA
Huatai HT cover 1mn shares 601988-C1 limit 5.43, POV 10% passive
Huatai HT cover 380.8K shares 6099-HK limit 14.8, POV 5% passive 
CLSA short 51K shares 1783-HK limit 7.8, POV 12% passive 
CLSA short 450K shares 1280-HK limit 2.53, POV 15% passive 
CLSA short 265K shares 2473-HK limit 7.0, POV 15% passive 
CICC SELL 1mn shares 600233-CH limit 13.6, POV 8% passive 
MACQ short 500K shares 2477-HK limit 22.75, NEO 10%-15% 
Maybank short 5mn shares 1114-HK limit 4.15, POV 10% passive 
Maybank BUY 500K shares 1209-HK limit 29.45, POV 6% passive 
JEFC BUY 1mn shares 291-HK limit 24.75, POV 5% passive 
JEFS BUY 100K shares 2156-HK limit 2.4, POV 8% passive
'''

In [ ]:
text2=text.replace("  "," ").replace(" shares","").replace(" limit","").replace(",","").replace("CLSA Desk","CLSA_HT").replace("DESK POV","DESK").replace("Desk POV","DESK").replace(" HT","_HT").replace(" (low-touch)","").replace("~","").replace('passive ','passive').replace('MACQ','MEEX').replace('NEO','POV').replace('below ','1-').replace('near touch','neartouch').replace('near-touch ','NearTouch').replace('near-touch','NearTouch').replace('Low','5-10%').replace('Mid','10-20%').replace('High','15-30%').replace('Maybank','MKEA').replace('Inline','POV')
tradelist=[trades.rstrip().split(" ") for trades in text2.split("\n") if trades != ""]
#tradelist

In [ ]:
cimblist=[ " ".join(x[1:]) for x in tradelist if "CIMB" in x]
for order in cimblist:
    print(order)

In [ ]:
header=['Broker','Side','Quantity','Ticker','Price','Strategy','Vol(%)','Style']

In [ ]:
def qty_conv(q):
    if q[-2:].lower() in ['mn','mm'] :
        qty=int(float(q[:-2])*1000000)
    elif q[-1].lower()=='k':
        qty=int(float(q[:-1])*1000)
    else:
        qty=int(q)
    
    return qty

In [ ]:
tradeframe=pd.DataFrame(tradelist,columns=header)
#tradeframe=pd.DataFrame(tradelist)
#tradeframe

In [ ]:
tradeframe['Ticker']=tradeframe['Ticker'].map(lambda x: x.replace("-"," ")+" EQUITY")
tradeframe['Quantity']=tradeframe['Quantity'].map(qty_conv)
tradeframe['Vol(%)']=tradeframe['Vol(%)'].map(lambda x: x.replace("%","") if x is not None else x)
tradeframe['Side']=tradeframe['Side'].map(lambda x:x.upper())
tradeframe['Side']=tradeframe['Side'].map({'SHORT':'SHRT','COVER':'COVR',"BUY":"BUY","SELL":'SELL'})
#tradeframe

In [ ]:
brokers=['CLSA','CLSA_HT','Huatai','Huatai_HT','CICC_HT','CICC','Haitong','Haitong_HT','MEEX','MKEA','JEFS','JEFC']

In [ ]:
tradeframe2=tradeframe[tradeframe['Broker'].isin(brokers)]
#tradeframe2
tradeframe2['Style']=tradeframe2.apply(lambda row: row['Style'].replace('passive','neartouch') if row['Broker']=='MEEX' and row['Style']=='passive' else row['Style'], axis=1)
tradeframe2

In [ ]:
tradeframe2.to_csv('trades_{}.csv'.format(tDay.strftime("%Y%m%d")),index=False)

In [ ]:
quoteList=quote.replace("Quote borrow: ","").replace("quote borrow: ","").replace(":","").replace("Haitong","HT").replace("Guangfa","GF").split("/ ")
#quoteList

In [ ]:
def quote(broker):    
    List=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if broker in x]
    String=broker+":\nHi team, could you quote borrow on "
    if broker=='Citi'or broker=='JEFF'or broker=='Tiger':
        List=[x for x in List if 'HK' in x]
    for x in List:
        String=String+x.strip()+", "
    String=String+"tks\n"
    String
    return String

In [ ]:
brokerList=['GF','HT','CLSA','CICC','Huatai','MACQ','MB','Citi','JEFF','Tiger']#'CICC',
for broker in brokerList:
    print(quote(broker))

In [ ]:
cimbList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if "CIMB" in x]
shares=[1000000]*len(cimbList)
df=pd.DataFrame(shares,cimbList,columns=['qty'])
#csv_path='borrow.csv'
#df.to_csv('borrow.csv')
#os.system(f'start excel "{csv_path}"') 
df

In [ ]:
mbList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if "MB" in x]
shares=[1000000]*len(mbList)
df=pd.DataFrame(shares,mbList,columns=['qty'])
#csv_path='borrow.csv'
#df.to_csv('borrow.csv')
#os.system(f'start excel "{csv_path}"') 
df

In [ ]:
macqList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if 'MACQ' in x]
shares=[1000000]*len(macqList)
macq=pd.DataFrame(shares,macqList,columns=['quantity'])
macq=macq.reset_index()
macq.columns=['ticker','quantity']
def symbol(ticker):
    if ticker[-2:]=="CH":
        if ticker[0]=="6":
            ticker=ticker.replace("CH","SH")
        else:
            ticker=ticker.replace("CH","ZK")
    if ticker[-2:]=="HK":
        ticker=ticker.zfill(7)
    ticker=ticker.replace("-",".")
    return ticker
macq['ticker']=macq['ticker'].map(lambda x: x.replace(" ",""))
macq['ticker']=macq['ticker'].map(lambda x: symbol(x))
macq.loc[macq.shape[0]]=["1211.HK",500000]
macq.loc[macq.shape[0]]=["0300.HK",500000]
macq.loc[macq.shape[0]]=["0921.HK",500000]
csv_path='macqborrow.csv'
macq.to_csv('macqborrow.csv',index=False)
#os.system(f'start excel "{csv_path}"') 
macq